# PRÁTICA GUIADA: Random Forest e ExtraTrees no Scikit Learn

## 1. Introdução
Nesta prática, vamos comparar o desempenho dos seguintes algoritmos:

-Árvores de decisão
-Bagging sobre Árvores de decisão
-Random Forest
-Extra Trees


Introdução aos métodos de combinação de modelos simples:

Todos os três são os chamados "meta-algoritmos": abordagens para combinar várias técnicas de aprendizado de máquina em um modelo preditivo, a fim de diminuir a variância (Bagging), o viés (boosting) ou melhorar a força preditiva (stacking).

Todo algoritmo consiste em duas etapas:

Produzir uma distribuição de modelos ML simples em subconjuntos dos dados originais.

Combinando a distribuição em um modelo "agregado".

Aqui está uma breve descrição de todos os três métodos:

**Bagging** (que significa **B**ootstrap **Agg**regat **ing** ) é uma maneira de diminuir a variância de sua previsão, gerando dados adicionais para formação do conjunto de dados original usando combinações com repetições para produzir multisets da mesma cardinalidade / tamanho de seus dados originais. Ao aumentar o tamanho do seu conjunto de treinamento, você não pode melhorar a força preditiva do modelo, mas apenas diminuir a variação, ajustando a previsão ao resultado esperado.

https://www.youtube.com/watch?v=sVriC_Ys2cw


**Boosting** é uma abordagem em duas etapas, em que primeiro usamos subconjuntos dos dados originais para produzir uma série de modelos de desempenho médio e, em seguida, "impulsionamos" seu desempenho combinando-os usando uma função de custo específica (por exemplo voto majoritário). Diferentemente do bagging, no boosting clássico, a criação do subconjunto não é aleatória e depende do desempenho dos modelos anteriores: cada novo subconjunto contém os elementos que (provavelmente seriam) incorretamente classificados pelos modelos anteriores.

O **stacking** é semelhante ao boosting: você também aplica vários modelos aos dados originais. A diferença aqui é, no entanto, que você não tem apenas uma fórmula para a sua função de custo, mas introduz um meta-nível e usa outro modelo/abordagem para estimar a entrada junto com as saídas de cada modelo para estimar os pesos ou, em outras palavras, para determinar quais modelos têm um bom desempenho e o que mal recebem esses dados de entrada. O pacote em python, MLENS, ajuda a implementar esse tipo de método que pode se tornar bastante complexo.

Aqui está uma tabela de comparação:

<img src='img/RFfqb.png'>

Como você vê, todas essas abordagens são diferentes para combinar vários modelos em um melhor, e não há um único vencedor aqui: tudo depende do seu domínio e do que você vai fazer. Você ainda pode tratar o stacking como uma espécie de boosting com um passo a mais, no entanto, a dificuldade de encontrar uma boa abordagem para o seu meta-nível dificulta a aplicação dessa abordagem na prática.


Na aula de hoje estamos interessados apenas em Bagging.

Vamos lá.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('cars.csv')
df.dtypes

buying           object
maint            object
doors            object
persons          object
lug_boot         object
safety           object
acceptability    object
dtype: object

Desta vez, vamos codificar as características usando um esquema de codificação One Hot, isto é, vamos considerá-las como variáveis categóricas. Também vamos codificar o target usando o `LabelEncoder`.

In [5]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [7]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

lab_enc = LabelEncoder()
lab_enc.fit(df['acceptability'])

LabelEncoder()

In [8]:
y = lab_enc.transform(df['acceptability'])
X = pd.get_dummies(df.drop('acceptability', axis=1))

X.iloc[:,0:8].head()

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh
0,0,0,0,1,0,0,0,1
1,0,0,0,1,0,0,0,1
2,0,0,0,1,0,0,0,1
3,0,0,0,1,0,0,0,1
4,0,0,0,1,0,0,0,1


Para que os resultados sejam consistentes, é necessário expor os modelos exatamente ao mesmo esquema de validação cruzada.

In [5]:
from sklearn.model_selection import cross_val_score,StratifiedKFold
cv = StratifiedKFold(n_splits=3, random_state=41, shuffle=True)

## 2. Comparando o desempenho das árvores de decisão e montagens de modelos
 
Agora vamos inicializar o classificador da árvore de decisão, avaliar seu rendimento e compará-lo com o desempenho das montagens que vimos até agora. Para isso, usaremos os seguintes métodos:

### RandomForestClassifier()

Este método implementa e executa uma RandomForest para resolver um problema de classificação. Alguns dos parâmetros mais importantes são os seguintes:

* `n_estimators`: o número de iterações (ou seja, de `base_estimators`) para treinar
* `criterion`: define o critério de impureza para avaliar a qualidade das partições (por padrão, é `gini`) 
* `max_features`: a quantidade de features que extrairá para treinar cada `base_estimator`. Por padrão, é igual a `sqrt(X.shape[1])`
* `bootstrap` e `bootstrap_features`: controla se tanto os n_samples como os features são extraídos com reposição.
* `max_depth`: a profundidade máxima da árvore
* `min_samples_leaf`: o número mínimo de n_samples para constituir uma folha da árvore (nó terminal)
* `min_samples_split`: o número mínimo de n_samples para realizar um split.

e vários outros que podem ser importantes no momento de realizar o tunning. Em geral, os mais importantes costumam ser: `n_estimators`, `max_features`, `max_depth` e `min_samples_leaf`.


### ExtraTreesClassifier()

Com este método é possível estimar um conjunto de árvores de decisão randomizadas. Usa os mesmos parâmetros que `RandomForestClassifier()`.


### BaggingClassifier()

Este método é muito interessante porque, ao contrário dos anteriores, é um "metaestimador", está situado em um nível maior de abstração. Ou seja, permite implementar o algoritmo de bagging (para classificação) com quase qualquer estimador Scikit-Learn. Usa como parâmetros análogos aos dois métodos anteriores (com diferentes valores por padrão em alguns casos). Os únicos "novos" são: 

* `base_estimator`: o estimador sobre o qual queremos executar o bagging (regressões, árvores, etc...)
* `max_samples`: a quantidade de n_samples de amostra em cada iteração. Por padrão, é igual a `sqrt(X.shape[0])`


Para comparar os diferentes algoritmos montamos a seguinte função. Usem como input um estimador e um string com o nome que queiram pôr e executem um `cross_val_score`

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier

def avaliar_desempenho(modelo, nome):
    s = cross_val_score(modelo, X, y, cv=cv, n_jobs=-1)
    print("Rendimento de {}:\t{:0.3} ± {:0.3}".format( \
        nome, s.mean().round(3), s.std().round(3)))
    
    
dt = DecisionTreeClassifier(class_weight='balanced')

avaliar_desempenho(dt,"Árvore de decisão")

Rendimento de Árvore de decisão:	0.964 ± 0.008


Tentem agora com os modelos restantes e avaliem o desempenho.  
 Bagging de Árvores de decisão
 * RandomForest
 *ExtraTrees

Recomendamos consultar a documentação para ver quais parâmetros são aceitos.   
http://scikit-learn.org/stable/modules/ensemble.html#forest

In [7]:
bdt = BaggingClassifier(DecisionTreeClassifier())
rf = RandomForestClassifier(class_weight='balanced')
et = ExtraTreesClassifier(class_weight='balanced')

avaliar_desempenho(dt,"Árvore de decisão")
avaliar_desempenho(bdt, "Bagging AD")
avaliar_desempenho(rf, "Random Forest")
avaliar_desempenho(et, "Extra Trees")

Rendimento de Árvore de decisão:	0.964 ± 0.006
Rendimento de Bagging AD:	0.969 ± 0.006
Rendimento de Random Forest:	0.951 ± 0.006
Rendimento de Extra Trees:	0.958 ± 0.005


Nesse caso, o bagging de árvores de decisão funciona melhor do que o resto.   
Com outros conjuntos de dados, os modelos Random Forest e Extra Trees podem ter resultados melhores e merecem ser testados. Poderíamos implementar um gridsearh para tentar realizar um tunning dos hiperparâmetros...

## 3. Tunando os hiperparâmetros de RandomForest

In [8]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
param_trees = {'n_estimators': [50, 100, 200],
               'max_features': [1, 5, 8, 10, 21],
               'max_depth': [5, 20, 50, 70, 100],
               'min_samples_leaf':[1, 5, 8, 10, 50]}

In [9]:
rf = RandomForestClassifier(class_weight='balanced')
kf = StratifiedKFold(n_splits=3, shuffle=True)

In [10]:
grid_search_rf = GridSearchCV(rf, param_grid=param_trees, cv=kf, verbose=1, n_jobs=3)

In [11]:
grid_search_rf.fit(X, y)

Fitting 3 folds for each of 375 candidates, totalling 1125 fits


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    3.8s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   11.2s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:   25.6s
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:   46.4s
[Parallel(n_jobs=3)]: Done 1125 out of 1125 | elapsed:  1.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'n_estimators': [50, 100, 200], 'max_features': [1, 5, 8, 10, 21], 'max_depth': [5, 20, 50, 70, 100], 'min_samples_leaf': [1, 5, 8, 10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [12]:
grid_search_rf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=70, max_features=10,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [13]:
grid_search_rf.best_score_

0.9762731481481481

É possível ver que, realizando um processo de tunnig, o RandomForest agora é o algoritmo que melhora o desempenho dos classificadores comparados.